# trio-parallel
* https://github.com/richardsheridan/trio-parallel
* https://trio-parallel.readthedocs.io/en/latest/reference.html

# *Status :-(*
*The `trio-parallel` example, which works fine as a python program, doesn't work in the notebook environment.*

In [1]:
%pip install -U trio-parallel

Note: you may need to restart the kernel to use updated packages.


In [2]:
%autoawait trio

In [3]:
import multiprocessing
import trio
import trio_parallel
import time

In [4]:
def hard_work(n, x):
    t = time.perf_counter() + n
    y = x
    while time.perf_counter() < t:
        x = not x
    print(y, "transformed into", x)
    return x


async def too_slow():
    await trio_parallel.run_sync(hard_work, 20, False, cancellable=True)


async def amain():
    t0 = time.perf_counter()
    async with trio.open_nursery() as nursery:
        nursery.start_soon(trio_parallel.run_sync, hard_work, 3, True)
        nursery.start_soon(trio_parallel.run_sync, hard_work, 1, False)
        nursery.start_soon(too_slow)
        result = await trio_parallel.run_sync(hard_work, 2, None)
        nursery.cancel_scope.cancel()
    print("got", result, "in", time.perf_counter() - t0, "seconds")

In [6]:
await amain()

Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3425, in run_code
    await eval(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-e11196ac7e3e>", line 1, in <module>
    await amain()
  File "<ipython-input-4-d0f932559a9a>", line 21, in amain
    nursery.cancel_scope.cancel()
  File "/opt/conda/lib/python3.8/site-packages/trio/_core/_run.py", line 815, in __aexit__
    raise combined_error_from_nursery
trio.MultiError: BrokenWorkerError("<SpawnProcess name='trio-parallel worker process 3' pid=3055 parent=3032 started daemon> died unexpectedly"), BrokenWorkerError("<SpawnProcess name='trio-parallel worker process 1' pid=3057 parent=3032 started daemon> died unexpectedly"), BrokenWorkerError("<SpawnProcess name='trio-parallel worker process 2' pid=3054 parent=3032 started daemon> died unexpectedly"), BrokenWorkerError("<SpawnProcess name='trio-parallel worker process 0' pid=3056 parent=3032 sta

In [ ]:
if __name__ == "__main__":
    multiprocessing.freeze_support()
    trio.run(amain)